# S3 submodule tutorial

Welcome to the S3 submodule tutorial. This tutorial will walk you through the basics of using the S3 submodule to interact with the risk S3 buckets.

## Introduction

Our data are stored in aws s3 buckets, which need to be accessed either using the aws cli or the boto3 python library. The S3 submodule provides a simple interface to the s3 buckets, allowing you to easily interact with the buckets using only python.

First, one needs to add temporary the directory containing the submodule to the path. This can be done by adding the following lines to the beginning of the notebook:

In [2]:
import sys

sys.path.append("../utils")
from s3 import S3Bucket, read_credentials

Then, we need to load your credentials. These should NEVER be directly written in your code.
Depending on your OS or the way you saved the credentials for the aws cli, the file "credentials" should be located in one of the following directories of the next cell. 

Please change the path accordingly if you are not using a linux machine or if you saved the credentials in a different directory.

In [3]:
if sys.platform == "win32":  # Windows
    credentials_path = r"C:\Users\Jules\.aws\\"  # Change Jules to your username
else:  # Linux or MacOS
    credentials_path = r"~/.aws/"

credentials = read_credentials(credentials_path)

Then, we can create a S3Bucket object and use it to upload and download files from S3 for instance.

By default, the bucket's name is "risk". 

In [4]:
bucket = S3Bucket(bucket_name="risk", **credentials)

## s3 submodule usage

This section is not exhaustive, but it should give you a good idea of how to use the S3Bucket class. For more information, please refer directly to the docstrings.

If you want to list folders in the bucket

In [5]:
_ = bucket.list_folders(prefix="", delimiter="/")
print("Example 1 :", _)
_ = bucket.list_folders(prefix=_[0], delimiter="/")
print("Example 2 :", _)

Example 1 : ['Old/', 'datacube/', 'datasets_grl/']
Example 2 : ['Old/ERA5Land/', 'Old/FWI/', 'Old/Historiques_feux/', 'Old/MODIS_LST/', 'Old/MODIS_NDVI/', 'Old/Old/', 'Old/Roads_distance/', 'Old/SAMPLE/', 'Old/SMI/', 'Old/Waterway_distance/', 'Old/Yearly_population/', 'Old/copernicus_elevation_and_slope/']


Knowing the folder structure of the bucket, we can now list the files in a folder. 

For example, we can list the files in the folder "Old/", with .csv or .txt in their name.

In [8]:
_ = bucket.list_files(
    prefix="Old/", patterns=[".csv", ".txt"]
)  # patterns can also be a part of the filename
print(len(_))

50


Following the same procedure, we can list files' metadata such as their size, last modified date. Or get all the metadata only file by file by using prefix for the whole file path.

In [9]:
_ = bucket.get_files_metadata(prefix="Old/", patterns=[".csv", ".txt"])
print("Example 1 :", _[0])
_ = bucket.get_files_metadata(prefix="Old/ERA5Land/2022.csv")
print("Example 2 :", _)

Example 1 : {'file_name': 'Old/ERA5Land/2022.csv', 'file_size': 611202.47, 'file_last_modified': datetime.datetime(2023, 9, 10, 17, 21, 47, tzinfo=tzutc())}
Example 2 : [{'file_name': 'Old/ERA5Land/2022.csv', 'file_size': 611202.47, 'file_last_modified': datetime.datetime(2023, 9, 10, 17, 21, 47, tzinfo=tzutc())}]


One can also donwload a whole folder from the bucket. The prefix will be used to filter the files to download. The whole architecture of the folder and its parent will be recreated locally. 

The ``save_path`` optional argument will be used if you need to save the folder in a specific location. Otherwise, the folder will be saved in the current working directory. Let's say if you need to have multiple versions of the same folder, you can use the ``save_path`` argument to save them in different locations.

In [ ]:
bucket.download_folder(prefix="Old/ERA5Land/", save_path="Version_1/")

Using only the path of a certain file and the name you want to give it, you can download a file with : 

In [ ]:
bucket.download_file("Old/ERA5Land/2022.csv", "2022.csv")

If needed, you need the key of the file to delete it. 
By the same way, you can upload a file by specifying its local file path and the key you want to assign. Keep in mind that the key is the path of the file in the bucket and the name you want to give it.

The functions are ``upload_file`` and ``delete_file``.